Ran on google colab

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.metrics import label_ranking_average_precision_score as LRAP

tqdm.pandas()

%matplotlib inline

SEED = 42

NUM_FEATURES = 5000
NUM_CLASSES = 3993

X_train = pd.read_csv("drive/My Drive/expanded/train_features.csv", names=range(NUM_FEATURES))
y_train = pd.read_csv("drive/My Drive/expanded/train_labels.csv", names=range(NUM_CLASSES))

X_val = pd.read_csv("drive/My Drive/expanded/dev_features.csv", names=range(NUM_FEATURES))
y_val = pd.read_csv("drive/My Drive/expanded/dev_labels.csv", names=range(NUM_CLASSES))

In [0]:
y_train = np.array(y_train)
y_val = np.array(y_val)

In [28]:
print('train labels:',X_train.shape)
print('validation labels:',X_val.shape)
print('train features:', y_train.shape)
print('validation features:',y_val.shape)

train labels: (15511, 5000)
validation labels: (1314, 5000)
train features: (15511, 3993)
validation features: (1314, 3993)


Really large differences in max and variance:

In [29]:
np.sort(np.var(X_train, axis = 0))

array([7.39786264e-02, 8.58397490e-02, 1.23233068e-01, ...,
       3.89117787e+04, 4.50693076e+04, 5.87975513e+04])

In [30]:
np.sort(np.max(X_train, axis = 0))

array([2.88664600e+00, 3.75861900e+00, 3.79371000e+00, ...,
       2.28867539e+04, 2.43905879e+04, 2.63541699e+04])

Single feature prediction:

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

rf = [RandomForestClassifier(class_weight = 'balanced') for i in range(NUM_CLASSES)]
y_pred = np.zeros(y_val.shape)

accuracy = [0 for i in range(NUM_CLASSES)]
precision = [0 for i in range(NUM_CLASSES)]
running_lrap = [0 for i in range(NUM_CLASSES)]

for i in range(NUM_CLASSES):
# for i in range(20):
    y_train_single = y_train[:,i]
    rf[i].fit(X_train, y_train_single)
    y_pred[:,i] = rf[i].predict(X_val)
        
    if i%10 == 0:
        current_lrap = LRAP(y_val[:,:i+1], y_pred[:,:i+1])
        print("Iteration "+str(i)+" LRAP: "+str(current_lrap))

Iteration 0 LRAP: 1.0
Iteration 10 LRAP: 0.9430953369309524
Iteration 20 LRAP: 0.9199554250924106
Iteration 30 LRAP: 0.8906339503445109
Iteration 40 LRAP: 0.8585449381891057
Iteration 50 LRAP: 0.8299907482018661
Iteration 60 LRAP: 0.8068574244579202
Iteration 70 LRAP: 0.7828400897521126
Iteration 80 LRAP: 0.7699320705789524
Iteration 90 LRAP: 0.7399778240531636
Iteration 100 LRAP: 0.723233921565675
Iteration 110 LRAP: 0.7160590099231622
Iteration 120 LRAP: 0.6991985459269615
Iteration 130 LRAP: 0.6744011045012714
Iteration 140 LRAP: 0.6683572483636843
Iteration 150 LRAP: 0.6520327516651512
Iteration 160 LRAP: 0.6437404017151364
Iteration 170 LRAP: 0.6322311120902179
Iteration 180 LRAP: 0.6164603836662907
Iteration 190 LRAP: 0.6125763137925719
Iteration 200 LRAP: 0.6022147948739338
Iteration 210 LRAP: 0.5840732334890659
Iteration 220 LRAP: 0.5671296439779502
Iteration 230 LRAP: 0.5580885090302877
Iteration 240 LRAP: 0.5545087420782513
Iteration 250 LRAP: 0.5489629339904047
Iteration 260

In [0]:
y_pred_df = pd.DataFrame(y_pred)


y_pred_df.to_csv('drive/My Drive/expanded/saved_rf_predictions.csv')

In [35]:
LRAP(y_val, y_pred)

0.20913459470773968

In [0]:
# Get probabilities
y_prob_pred = np.zeros(y_val.shape)

for i in range(NUM_CLASSES):
    prob_predictions = rf[i].predict_proba(X_val)
    if prob_predictions.shape[1] == 2:
      y_prob_pred[:,i] = prob_predictions[:,1]

In [53]:
LRAP(y_val, y_prob_pred)

0.5638719823802654

In [0]:
y_prob_pred = pd.DataFrame(y_prob_pred)


y_prob_pred.to_csv('drive/My Drive/expanded/saved_rf_probabilities.csv')

In [56]:
y_prob_pred

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3953,3954,3955,3956,3957,3958,3959,3960,3961,3962,3963,3964,3965,3966,3967,3968,3969,3970,3971,3972,3973,3974,3975,3976,3977,3978,3979,3980,3981,3982,3983,3984,3985,3986,3987,3988,3989,3990,3991,3992
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Below run locally

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.metrics import label_ranking_average_precision_score as LRAP

tqdm.pandas()

%matplotlib inline

SEED = 42

NUM_FEATURES = 5000
NUM_CLASSES = 3993

raw_rf_predict = pd.read_csv("../data/results/rf-single-label/saved_rf_predictions.csv")
raw_rf_prob = pd.read_csv("../data/results/rf-single-label/saved_rf_probabilities.csv")

X_train = pd.read_csv("../data/expanded/train_features.csv", names=range(NUM_FEATURES))
y_train = pd.read_csv("../data/expanded/train_labels.csv", names=range(NUM_CLASSES))

X_val = pd.read_csv("../data/expanded/dev_features.csv", names=range(NUM_FEATURES))
y_val = pd.read_csv("../data/expanded/dev_labels.csv", names=range(NUM_CLASSES))

C:\Users\Andrew\Anaconda3\lib\site-packages\tqdm\std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
print(raw_rf_predict.shape)
print(raw_rf_prob.shape)
print(y_val.shape)

(1314, 3994)
(1314, 3994)
(1314, 3993)


Extra column since the first column is the index. Forgot to save with index off

In [4]:
np.max(raw_rf_prob, axis = 0)

Unnamed: 0    1313.0
0                0.0
1                1.0
2                0.0
3                1.0
               ...  
3988             0.0
3989             0.0
3990             0.0
3991             0.0
3992             0.0
Length: 3994, dtype: float64

In [5]:
rf_predict = np.array(raw_rf_predict)[:,1:]
rf_prob = np.array(raw_rf_prob)[:,1:]
y_val = np.array(y_val)

In [10]:
np.nanmean(np.sum(np.multiply(rf_predict == y_val, rf_predict == 1), axis = 1)/np.sum(rf_predict == 1, axis = 1))

C:\Users\Andrew\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


0.8354815864022661

In [11]:
# Evaluation metrics on the predictions instead of the probabilities
print("Accuracy:", np.sum(rf_predict == y_val)/rf_predict.size)
print("Precision:", np.nanmean(np.sum(np.multiply(rf_predict == y_val, rf_predict == 1), axis = 1)/np.sum(rf_predict == 1, axis = 1)))
print("Recall:", np.sum(np.multiply(rf_predict == y_val, y_val == 1))/np.sum(y_val == 1))


Accuracy: 0.9989138145483668
Precision: 0.8354815864022661
Recall: 0.22858372793801118


C:\Users\Andrew\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])